In [1]:
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
from tqdm.autonotebook import tqdm
#import pandas as pd
import mysql.connector
import codecs
codecs.register(lambda name: codecs.lookup('utf8') if name == 'utf8mb4' else None)
import re
import conllu

In [2]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd='password',
  database='commits_db',
  charset='utf8mb4')

cur = db.cursor()

In [3]:
def yield_pos_short():
    #mycursor.execute('SELECT conllu, language FROM C LEFT JOIN R ON C.repo_id=R.gh_id LIMIT 1000;')
    cur.execute('SELECT conllu FROM C')
    text = cur.fetchone()
    while text:
        #doc = conllu.parse(text[0])
        doc = re.findall('sent_id', text[0])
        if len(doc) == 1:
            doc = conllu.parse(text[0])
            if len(doc[0].tokens) <= 20:
                yield ' '.join([i['upostag'] for i in doc[0].tokens]), ' '.join([i['deprel'] for i in doc[0].tokens])
        text = cur.fetchone()

In [4]:
file = open('/home/glznv/pos_short_commits2.txt', 'w')
file2 = open('/home/glznv/rel_short_commits2.txt', 'w')
for text in tqdm(yield_pos_short()):
    print(text[0], file=file)
    print(text[1], file=file2)

In [5]:
def yield_2plus_short(first, latter):
    #mycursor.execute('SELECT conllu, language FROM C LEFT JOIN R ON C.repo_id=R.gh_id LIMIT 1000;')
    cur.execute('SELECT conllu FROM C')
    text = cur.fetchone()
    while text:
        #doc = conllu.parse(text[0])
        doc = re.findall('sent_id', text[0])
        if len(doc) > 1:
            doc = conllu.parse(text[0])
            if len(doc[0]) > 1:
                f = ' '.join([i['deprel'] for i in doc[0].tokens])
                s = '\n'.join([' '.join([i['deprel'] for i in d.tokens]) for d in doc[1:]])
                print(f, file=first)
                print(s, file=latter)
                yield 1
        text = cur.fetchone()

In [6]:
first = open('/home/glznv/2plus_0.txt', 'w')
latter = open('/home/glznv/2plus_next.txt', 'w')
for text in tqdm(yield_2plus_short(first, latter)):
    pass